In [3]:
import os
import time
import threading
import librosa
import numpy as np
from sklearn.tree import DecisionTreeRegressor  # Change to DecisionTreeRegressor for regression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

model = None

# Function to extract tempo from audio file using librosa
def extract_tempo(audio_path):
    y, sr = librosa.load(audio_path)
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)
    return tempo

# Function to train the machine learning model with cross-validation
def train_machine_learning_model():
    # Your file paths for training data here
    original_song_paths = [r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 1 - 1.2.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 3 - 1.3.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 4 - 1.5.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 5 - 1.3.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 6 - 1.1.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 7 - 1.2.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 8 - 1.4.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 9 - 1.6.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 10 - 1.2.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 11 - 1.5.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 12 - 1.7.mp3']
    rhythm_paths = [r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 1_1.2.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 3_1.3.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 4_1.5.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 5_1.3.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 6_1.1.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 7_1.2.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 8_1.4.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 9_1.6.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 10_1.2.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 11_1.5.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 12_1.7.mp3']

    original_song_tempos = [extract_tempo(audio_path) for audio_path in original_song_paths]
    rhythm_tempos = [extract_tempo(audio_path) for audio_path in rhythm_paths]

    global X, Y
    X = np.array(original_song_tempos + rhythm_tempos).reshape(-1, 1)
    Y = [0] * len(original_song_tempos) + [1] * len(rhythm_tempos)

    X = np.array(X)
    Y = np.array(Y)

    global model
    model = DecisionTreeRegressor()  # Change to DecisionTreeRegressor for regression

    # Perform cross-validation
    scores = cross_val_score(model, X, Y, cv=5, scoring='neg_mean_squared_error')

    print("Cross-validation mean squared errors:", -scores)
    print("Mean cross-validation mean squared error:", -scores.mean())

    model.fit(X, Y)

# Function to evaluate the model
def evaluate_model():
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    model.fit(X_train, Y_train)

    Y_pred = model.predict(X_test)

    mse = mean_squared_error(Y_test, Y_pred)
    print(f"Mean Squared Error: {mse}")

# Main function
def main():
    start_time = time.time()
    train_machine_learning_model()
    end_time = time.time()

    evaluate_model()

if __name__ == "__main__":
    main()


Cross-validation mean squared errors: [0.52177778 0.71388889 0.37472222 0.51361111 0.56222222]
Mean cross-validation mean squared error: 0.5372444444444444
Mean Squared Error: 0.5152777777777778


In [4]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import tkinter as tk
from tkinter import filedialog
from flask import Flask, render_template_string
import os
import threading
import time
from sklearn.tree import DecisionTreeRegressor
import librosa
import numpy as np

app = Flask(__name__)

model = None

# Function to save predicted tempo and meter to a single text file
def save_prediction_to_text(predicted_tempo, predicted_meter, output_path):
    with open(output_path, 'w') as file:
        file.write(f"Predicted Music Tempo: {predicted_tempo} BPM\n")
        file.write(f"Predicted Music Meter: {predicted_meter} seconds\n")

# Function to browse for a file
def browse_file():
    file_path = filedialog.askopenfilename(filetypes=[("Audio Files", ".mp3;.wav")])
    return file_path if file_path else None

# Route for downloading text
@app.route('/download_text')
def download_text():
    return render_template_string('Download your text file content here.')

# Function to extract tempo from audio file using librosa
def extract_tempo(audio_path):
    y, sr = librosa.load(audio_path)
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)
    return tempo

# Function to extract meter (rhythm) from audio file
def extract_meter(audio_path):
    y, sr = librosa.load(audio_path)
    onset_frames = librosa.onset.onset_detect(y=y, sr=sr)
    intervals = np.diff(onset_frames)
    meter = np.median(intervals)
    return meter

# Function to train the machine learning model with cross-validation
def train_machine_learning_model():
    # Your file paths for training data here
    original_song_paths = [r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 1 - 1.2.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 3 - 1.3.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 4 - 1.5.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 5 - 1.3.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 6 - 1.1.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 7 - 1.2.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 8 - 1.4.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 9 - 1.6.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 10 - 1.2.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 11 - 1.5.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Input/Mix 12 - 1.7.mp3']
    rhythm_paths = [r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 1_1.2.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 3_1.3.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 4_1.5.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 5_1.3.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 6_1.1.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 7_1.2.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 8_1.4.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 9_1.6.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 10_1.2.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 11_1.5.mp3', r'C:/Users/mrudu/OneDrive/Desktop/Output/Mix 12_1.7.mp3']


    original_song_tempos = [extract_tempo(audio_path) for audio_path in original_song_paths]
    rhythm_tempos = [extract_tempo(audio_path) for audio_path in rhythm_paths]

    global X, Y
    X = np.array(original_song_tempos + rhythm_tempos).reshape(-1, 1)
    Y = [0] * len(original_song_tempos) + [1] * len(rhythm_tempos)

    X = np.array(X)
    Y = np.array(Y)

    global model
    model = DecisionTreeRegressor()
    model.fit(X, Y)

# Function to predict the meter of an audio file
def predict_meter(audio_path):
    try:
        meter = extract_meter(audio_path)
        return meter
    except Exception as e:
        print(f"An error occurred while predicting meter: {e}")
        return None

# Function to test the trained model
def test_model(file_path):
    try:
        predicted_tempo = model.predict([[extract_tempo(file_path)]])[0]
        predicted_meter = predict_meter(file_path)  # Predict meter
        print(f"Predicted tempo: {predicted_tempo} BPM")
        print(f"Predicted meter: {predicted_meter} seconds")
        return predicted_tempo, predicted_meter
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

# Function to run the Flask app
def run_flask_app():
    app.run()

# Main function
def main():
    root = tk.Tk()

    train_machine_learning_model()

    file_path = browse_file()

    if file_path:
        try:
            current_directory = os.path.abspath(os.getcwd())
            output_directory = os.path.join(current_directory, "output")
            os.makedirs(output_directory, exist_ok=True)

            predicted_tempo, predicted_meter = test_model(file_path)

            if predicted_tempo is not None and predicted_meter is not None:
                output_text_path_tempo = os.path.join(output_directory, "predicted_tempo_meter.txt")

                save_prediction_to_text(predicted_tempo, predicted_meter, output_text_path_tempo)

                print(f"Predicted tempo and meter saved as text: {output_text_path_tempo}")

                flask_thread = threading.Thread(target=run_flask_app)
                flask_thread.start()

                root.destroy()
                time.sleep(2)

                print("You can download the predicted tempo and meter text file from:")
                print(f"http://127.0.0.1:5000/download_text")

        except Exception as e:
            print(f"An error occurred: {e}")
    else:
        print("No file selected.")

if __name__ == "__main__":
    main()









Predicted tempo: 0.5 BPM
Predicted meter: 7.0 seconds
Predicted tempo and meter saved as text: C:\Users\mrudu\output\predicted_tempo_meter.txt
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


You can download the predicted tempo and meter text file from:
http://127.0.0.1:5000/download_text
